In [3]:
import os, json, math, numpy as np, pandas as pd
from pathlib import Path

In [4]:
DATA_DIR = Path("../data")
IN_PATH = DATA_DIR / "pairwise.jsonl"
OUT_JSON = DATA_DIR / "elo_ratings.json"
OUT_CSV = DATA_DIR / "elo_ratings.csv"

PRIOR_STD = float(os.environ.get("PRIOR_STD","350"))
PRIOR_MEAN = 1500.0
TARGET_MEAN = 1500.0
MAX_ITERS = 2000
TOL = 1e-7
LR = 100.0

In [5]:
rows=[]
with open(IN_PATH,"r",encoding="utf-8") as f:
    for line in f:
        line=line.strip()
        if line:
            rows.append(json.loads(line))
ids = sorted(list({r["left_id"] for r in rows} | {r["right_id"] for r in rows}))
idx = {k:i for i,k in enumerate(ids)}
n=len(ids)

wins = np.zeros((n,n),dtype=np.float64)
counts = np.zeros((n,n),dtype=np.float64)
for r in rows:
    i = idx[r["left_id"]]; j = idx[r["right_id"]]
    counts[i,j]+=1; counts[j,i]+=1
    if r["winner"]==1:
        wins[i,j]+=1
    else:
        wins[j,i]+=1

pairs=[]
for i in range(n):
    for j in range(i+1,n):
        n_ij = counts[i,j]
        if n_ij>0:
            w_ij = wins[i,j]
            pairs.append((i,j,w_ij,n_ij))

W = np.sum(wins,axis=1)
L = np.sum(wins,axis=0)

In [6]:
k = math.log(10.0)/400.0

def ll_and_grad(r):
    g = np.zeros_like(r)
    ll = 0.0
    for i,j,w,n_ij in pairs:
        pij = 1.0/(1.0+10.0**((r[j]-r[i])/400.0))
        pij = min(max(pij,1e-12),1-1e-12)
        ll += w*math.log(pij) + (n_ij-w)*math.log(1.0-pij)
        d = k*(w - n_ij*pij)
        g[i] += d
        g[j] -= d
    var = PRIOR_STD**2
    ll += -0.5*np.sum((r-PRIOR_MEAN)**2)/var
    g += -(r-PRIOR_MEAN)/var
    return ll, g

r = np.full(n, PRIOR_MEAN, dtype=np.float64)
best_ll, best_r = -1e300, r.copy()

In [7]:
for it in range(MAX_ITERS):
    ll, g = ll_and_grad(r)
    step = LR
    ok = False
    for _ in range(20):
        r_new = r + step*g
        ll_new, _ = ll_and_grad(r_new)
        if ll_new > ll:
            r = r_new
            ll = ll_new
            ok = True
            break
        step *= 0.5
    if not ok:
        break
    if ll > best_ll:
        best_ll, best_r = ll, r.copy()
    if np.linalg.norm(step*g, ord=np.inf) < TOL:
        break

r = best_r - np.mean(best_r) + TARGET_MEAN
games = (W+L).astype(int)
df = pd.DataFrame({"id":ids,"rating":r,"wins":W.astype(int),"losses":L.astype(int),"games":games}).sort_values("rating",ascending=False).reset_index(drop=True)

OUT_JSON.write_text(json.dumps(
    [{"id":row.id,"rating":float(row.rating),"wins":int(row.wins),"losses":int(row.losses),"games":int(row.games)} for row in df.itertuples(index=False)],
    ensure_ascii=False, indent=2
), encoding="utf-8")
df.to_csv(OUT_CSV,index=False)
print(str(OUT_JSON))
print(str(OUT_CSV))
print(f"mean={df.rating.mean():.3f} std={df.rating.std(ddof=0):.3f}")

../data/elo_ratings.json
../data/elo_ratings.csv
mean=1500.000 std=193.164
